In [ ]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def go_to_page(page_num):
    # Wait for the page box to become visible
    page_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "pagebox")))
    
    i = 1
    while True:
        try:
            # Find and click on the page link
            page = page_box.find_element(By.LINK_TEXT, str(page_num))
            page.click()
            time.sleep(1)
            break
        except:
            i += 4
            # Find and click on the page link
            page = page_box.find_element(By.LINK_TEXT, str(i))
            page.click()
            time.sleep(1)

In [ ]:
driver = webdriver.Chrome()
driver.get("https://navi.cnki.net/knavi/newspapers/RMRB/detail?uniplatform=NZKPT")

search_box = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "J_searchTxt"))
)

# 在输入框中输入文字
search_box.send_keys("疫情")
time.sleep(2)

# press enter
search_box.send_keys(Keys.ENTER)
time.sleep(1)
sort_dropdown = driver.find_element(By.CLASS_NAME, "sort_select_default")

# hover over the sorting options dropdown
ActionChains(driver).move_to_element(sort_dropdown).perform()

# click and then select the date option
date_option = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//a[@name="DT"]'))
)
date_option.click()
time.sleep(1)

go_to_page(1)

In [ ]:
# Find the <tbody> element within the table (assuming there is only one table)
tbody = driver.find_element(By.XPATH, "//table/tbody")

# Find all <tr> elements within the <tbody>
rows = tbody.find_elements(By.TAG_NAME, "tr")
for row in rows:
    link = row.find_element(By.XPATH, ".//td[@class='name']/a").get_attribute("href")
    # click on the link
    driver.get(link)
    html_link = driver.find_element(By.CLASS_NAME, "btn-html")
    html_link.click()
    time.sleep(1.1)
    break

# Please log in through WeChat, and then run the following code

In [ ]:
# go to the new tab
driver.switch_to.window(driver.window_handles[1])
driver.close()
driver.switch_to.window(driver.window_handles[0])
driver.back()

In [ ]:
# download the page
def download_page(page_num):
    go_to_page(page_num)
    
    # Find the <tbody> element within the table (assuming there is only one table)
    tbody = driver.find_element(By.XPATH, "//table/tbody")
    # Find all <tr> elements within the <tbody>
    rows = tbody.find_elements(By.TAG_NAME, "tr")

    links = []
    contents = []
    keywords = []
    for row in rows:
        link = row.find_element(By.XPATH, ".//td[@class='name']/a").get_attribute("href")
        links.append(link)
        driver.get(link)
        keyword = driver.find_element(By.CLASS_NAME, "keywords").text
        keywords.append(keyword)
        time.sleep(1.2)
        html_link = driver.find_element(By.CLASS_NAME, "btn-html")
        html_link.click()
        time.sleep(1.5)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(0.5)
        elements = driver.find_elements(By.XPATH, ".//*[contains(@class, 'p1') or contains(@class, 'anchor-tag')]")
        text = ""
        for element in elements:
            text += element.text
            text += "\n"
        contents.append(text)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.back()
        time.sleep(1)

    tbody_text = tbody.text
    tbody_text = tbody_text.split("\n")

    for i in range(len(tbody_text)):
        if i % 2 == 0:
            # split by the first space
            tbody_text[i] = tbody_text[i].split(" ", 1)
        else:
            if ";" not in tbody_text[i]:
                for k in range(len(tbody_text[i])):
                    if tbody_text[i][k].isnumeric():
                        tbody_text[i] = tbody_text[i][k:]
                        tbody_text[i] = 'NA; ' + tbody_text[i]
                        break
            tbody_text[i] = tbody_text[i].split(" ")
            if len(tbody_text[i]) == 3:
                tbody_text[i].append('0')

    # combine the odd and even rows to a large list
    tbody_text = [item for sublist in tbody_text for item in sublist]
    tbody_text = np.array(tbody_text).reshape(-1, 6)
    tbody_text = pd.DataFrame(tbody_text, columns=["Number", "Title", "Author", "banHao", "Date", "Download"])
    tbody_text["Links"] = links
    tbody_text["Contents"] = contents
    tbody_text["Keywords"] = keywords
    page_df = tbody_text.copy()
    return page_df

# 下面的代码可以根据自己需求调整

In [ ]:
start = 21
end = 31
for i in range (start, end + 1):
    df_current = download_page(i)
    df_current.to_csv("PD" + str(i) + ".csv", index=False, encoding="utf-8-sig")
    if i == start:
            df = df_current.copy()
    else:
        df = pd.concat([df, df_current], ignore_index=True)

In [ ]:
# 根据日期筛选
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(by="Date")
df = df[(df["Date"] >= "2022-04-01") & (df["Date"] <= "2022-06-01")]
df = df.reset_index(drop=True)
df.to_csv("data.csv", index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd
import os

# Define the directory where the CSV files are located
directory = '/Users/karan/Desktop/昆山杜克/SW/CNKI_Crawler-main'  # Replace with the path to your directory containing the CSV files

# Initialize an empty list to hold the DataFrames
dfs = []

# Loop over the range of file names
start = 15
end = 31
for i in range(start, end + 1):
    file_name = f"PD{i}.csv"
    file_path = os.path.join(directory, file_name)
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Read the CSV file and append it to the list of DataFrames
        df_current = pd.read_csv(file_path, encoding="utf-8-sig")
        dfs.append(df_current)
    else:
        print(f"{file_name} does not exist.")

# Concatenate all DataFrames in the list
df_merged = pd.concat(dfs, ignore_index=True)

# Write the merged DataFrame to a new CSV file
output_file_path = os.path.join(directory, '人民日报.csv')
df_merged.to_csv(output_file_path, index=False, encoding="utf-8-sig")

print(f"Merged CSV file has been saved to {output_file_path}")


In [ ]:
# Define the path to the merged CSV file
merged_file_path = '/Users/karan/Desktop/昆山杜克/SW/CNKI_Crawler-main/人民日报.csv'  # Replace with the path to your merged CSV file

# Define the path to save the filtered CSV file
output_file_path = '/Users/karan/Desktop/昆山杜克/SW/CNKI_Crawler-main/人民日报.csv'  # Replace with the path to save the filtered CSV file

# Define the date range
start_date = '2022-04-01'  # YYYY-MM-DD format
end_date = '2022-06-01'  # YYYY-MM-DD format

# Check if the merged file exists
if os.path.exists(merged_file_path):
    # Read the merged CSV file
    df_merged = pd.read_csv(merged_file_path, encoding="utf-8-sig")
    
    # Convert the 'date' column to datetime
    df_merged['Date'] = pd.to_datetime(df_merged['Date'], errors='coerce')
    
    # Filter the data based on the date range
    df_filtered = df_merged[(df_merged['Date'] >= start_date) & (df_merged['Date'] <= end_date)]
    
    # Write the filtered DataFrame to a new CSV file
    df_filtered.to_csv(output_file_path, index=False, encoding="utf-8-sig")
    
    print(f"Filtered CSV file has been saved to {output_file_path}")
else:
    print(f"{merged_file_path} does not exist.")

In [ ]:
pip install transformers torch


In [2]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
import textwrap
from tqdm import tqdm

# Load the model and tokenizer
model_name = "Helsinki-NLP/opus-mt-zh-en"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

def translate_long_text(text, max_length=512):
    """Translate text that might be longer than the model's maximum length."""
    chunks = textwrap.wrap(text, width=max_length, break_long_words=False, replace_whitespace=False)
    translated_chunks = [translate(chunk) for chunk in chunks]
    return " ".join(translated_chunks)

def translate(text):
    """Translate Chinese text to English using the pre-trained model."""
    tokenized_text = tokenizer.encode(text, return_tensors="pt")
    translated_tokens = model.generate(tokenized_text)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

# Load your dataset
input_path = '/Users/karan/Desktop/昆山杜克/SW/CNKI_Crawler-main/df15.csv'
output_path = '/Users/karan/Desktop/昆山杜克/SW/CNKI_Crawler-main/df15.csv'

df = pd.read_csv(input_path)

# Translate the 'content' column with progress visualization
df['content_en'] = [translate_long_text(text) for text in tqdm(df['Contents'], desc="Translating")]

# Save the updated dataframe
df.to_csv(output_path, index=False)


/opt/anaconda3/envs/CNKI/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Translating: 100%|██████████| 20/20 [16:08<00:00, 48.41s/it]


In [16]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.
